**Logistics Regression Model will be used for prediction**

Dataset Source: https://www.kaggle.com/c/fake-news/data?select=train.csv 

In [1]:
from google.colab import files
uploaded=files.upload()

About the dataset





1.   id:Unique ID for a news article
2.   title:Title of the news article
3.   author: Author of the news article
4.   text: Text of the article that can be incomplete
5.   label: A label that marks whether the news article real or fake(1: Fake  0: real)







Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re  #for searching word in a document
from nltk.corpus import stopwords  #to remove the stopwords(is,a,what...)
from nltk.stem.porter import PorterStemmer  #for removing prefix and sufix of a word and give back the root word
from sklearn.feature_extraction.text import TfidfVectorizer  #used for convert the text into feature vector
from sklearn.model_selection import train_test_split #split the dataset into training data and test data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')   #to donwload all the stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))  #To show all the stopwords in english language

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [ ]:
#loading the dataset into pandas data frame
news_dataset=pd.read_csv('/content/train.csv')

In [ ]:
#check number of rows and columns in the dataset
news_dataset.shape

(20800, 5)

In [ ]:
#print first 5 rows of the data frame
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#replacing the null values with empty string
news_dataset=news_dataset.fillna('')

In [ ]:
#merging the author name and news title to consider these two columns for our prediction
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [ ]:
print(news_dataset['content'])  #we will use this content data and label for prediction

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
#separating the data & label
X=news_dataset.drop(columns='label',axis=1)  #we will drop the label column from the dataset
Y=news_dataset['label']  # store it in Y

In [ ]:
print(X)
print(Y)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20795  20795  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20798  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  20799  ...            David Swanson What Keeps the F-35 Alive

[20800 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

Stemming: 

Stemming is process to reduce a word to its root word

Example:

actor,actress,acting......-> root word -> act

In [ ]:
port_stem=PorterStemmer()  #PorterStemmer is loaded to port_stem

In [ ]:
#function for stemming
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)   #remove all the things except the alphabets from the content and replace by space
  stemmed_content=stemmed_content.lower()  #convert all the upper case to lower case
  stemmed_content=stemmed_content.split()   #split all the words and store these into list
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]  #perform stemming for all the word if those word are not available in stopwords
  stemmed_content=' '.join(stemmed_content)   #join the words
  return stemmed_content

In [ ]:
#apply the stemming function in our dataset
news_dataset['content']=news_dataset['content'].apply(stemming)  #we are appling stemming function in content column of our dataset

In [ ]:
print(news_dataset['content'])   #print the processed and stemmed content column

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
#separating the data and label
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
X.shape

(20800,)

In [ ]:
Y.shape

(20800,)

In [ ]:
#all the content is in text format but the computer can not understand the text format
#so we have to convert text to some meaningful numbers so that computer can understand
#we will use TfidfVectorizer for this
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splitting the dataset into training and test data:

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)  #80% data go for train and 20% go for test

**Training The Model: Logistics Regression Model**

In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Evaluation

Accuracy Score

In [ ]:
#accuracy score on training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [ ]:
print("The Accuracy Score of the Training Data is: ",training_data_accuracy)

The Accuracy Score of the Training Data is:  0.9865985576923076


In [ ]:
#accuracy score on test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [ ]:
print("The Accuracy Score of the Test Data is: ",test_data_accuracy)

The Accuracy Score of the Test Data is:  0.9790865384615385


Making a Predictive System

In [ ]:
X_new=X_test[0]
prediction=model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('The news is real')
else:
  print('The news is fake')

[1]
The news is fake


In [ ]:
print(Y_test[0])

1


In [ ]:
X_new=X_test[1]
prediction=model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('The news is real')
else:
  print('The news is fake')

[0]
The news is real


In [ ]:
print(Y_test[1])

0
